In [432]:
#from transformers import pipeline
#from transformers import AutoTokenizer, AutoModelForSequenceClassification
#from transformers.pipelines.pt_utils import KeyDataset
#from datasets import load_dataset
import matplotlib.pyplot as plt
import torch.nn.functional as F
import io
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import glob
import os
import difflib
import re
import datetime

# Category and price clean up

In [228]:
# Get categories and price for each product
product_category = pd.read_excel("../Amazon.co.uk Best Sellers_unique_item_list.xlsx")

In [229]:
product_category

,Title,aiconrow__acontains_class_alinknormal_URL,aiconalt,asizesmall,Price,Price1,Unnamed: 6
0,Andrex Gentle Clean Toilet Rolls - 45 Toilet R...,https://www.amazon.co.uk/product-reviews/B07R4...,4.7 out of 5 stars,43441.0,£47.98,£21.09,NaN
1,Tassimo Kenco Americano Grande Coffee Pods (Pa...,https://www.amazon.co.uk/product-reviews/B08MR...,4.8 out of 5 stars,31717.0,£41.97,£15.76,NaN
2,DYLON Washing Machine Fabric Dye Pod for Cloth...,https://www.amazon.co.uk/product-reviews/B078W...,4.5 out of 5 stars,1578.0,£16.81,£5.00,NaN
3,Tassimo Costa Gingerbread Latte Coffee Pods (P...,https://www.amazon.co.uk/product-reviews/B09FF...,4.8 out of 5 stars,29274.0,£59.97,£22.48,NaN
4,Lindt Lindor Milk Chocolate Truffles Box with ...,https://www.amazon.co.uk/product-reviews/B08S8...,4.7 out of 5 stars,49431.0,£13.99,£3.99,NaN
...,...,...,...,...,...,...,...
2251,Nuby Fun Watering Can Bath Toy,https://www.amazon.co.uk/product-reviews/B06X1...,4.8 out of 5 stars,5233.0,NaN,£5.00,NaN
2252,NaN,NaN,NaN,NaN,£18.79,£1.87,NaN
2253,NaN,NaN,NaN,NaN,NaN,NaN,26226478.0
2254,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [230]:
product_category.drop(columns=["Unnamed: 6", "aiconalt", "asizesmall", "Price"], inplace= True)
product_category.dropna(subset=["Title"], inplace=True)
product_category.rename(columns={"aiconrow__acontains_class_alinknormal_URL": "Category", "Price1": "Price", "Title":"Product"}, inplace=True)

In [231]:
product_category['Category'] = product_category['Category'].str.rsplit('zg_bs_', 1).str.get(1)
product_category['Category'] = product_category['Category'].str.rsplit('_cr_', 1).str.get(0)

In [232]:
product_category["Price"] = product_category["Price"].astype(str).apply(lambda x: x.replace('£',''))

In [233]:
product_category

,Product,Category,Price
0,Andrex Gentle Clean Toilet Rolls - 45 Toilet R...,grocery,21.09
1,Tassimo Kenco Americano Grande Coffee Pods (Pa...,grocery,15.76
2,DYLON Washing Machine Fabric Dye Pod for Cloth...,grocery,5.00
3,Tassimo Costa Gingerbread Latte Coffee Pods (P...,grocery,22.48
4,Lindt Lindor Milk Chocolate Truffles Box with ...,grocery,3.99
...,...,...,...
2247,Sea Monkeys DI23230 Sea Monkeys Magic Castle,kids,7.50
2248,Unique Party 9054 - Blue Hair Spray,kids,2.45
2249,Nuby Octopus Floating Bath Toy,kids,8.25
2250,LEGO 43201 Disney Isabela’s Magical Door Build...,kids,17.99


# Clean review dataset

In [234]:
# setting the path for joining multiple files
files = os.path.join("../Raw(uncleaned) csv files", "Amazon.co.uk_ Best_All_Reviews***.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
print(df)

Resultant CSV after joining all CSV files at a particular location...
                                                      Text  \
0                        Andrex Gentle Clean Toilet Tissue   
1                        Andrex Gentle Clean Toilet Tissue   
2                        Andrex Gentle Clean Toilet Tissue   
3                        Andrex Gentle Clean Toilet Tissue   
4                        Andrex Gentle Clean Toilet Tissue   
...                                                    ...   
1019995  Garnier Hand Repair Intensive Restoring Hand C...   
1019996  Linda's Silicone Stove Gap Covers (2 Pack), He...   
1019997  Linda's Silicone Stove Gap Covers (2 Pack), He...   
1019998  Linda's Silicone Stove Gap Covers (2 Pack), He...   
1019999  Linda's Silicone Stove Gap Covers (2 Pack), He...   

                                                     Title            Name  \
0                             Fun Size C Wafer Thin Andrex        DM Bunny   
1             They've shrunk.

In [235]:
df.tail()

,Text,Title,Name,aiconalt,View,asizemini,State,View1,asizebase
1019995,Garnier Hand Repair Intensive Restoring Hand C...,Nothing special,Janice,3.0 out of 5 stars,Reviewed in the United Kingdom on 3 February 2017,NaN,Verified Purchase,"\n Very ordinary product. Initially, the han...",NaN
1019996,"Linda's Silicone Stove Gap Covers (2 Pack), He...",NaN,Tina,5.0 out of 5 stars,Reviewed in the United States on 9 January 2019,"Size Name: 21""Colour Name: Black",Verified Purchase,\n It came really fast. We have some difficul...,NaN
1019997,"Linda's Silicone Stove Gap Covers (2 Pack), He...",NaN,Mainegirl,5.0 out of 5 stars,Reviewed in the United States on 2 February 2020,"Size Name: 21""Colour Name: Black",Verified Purchase,\n Very happy with this purchase. Just what I...,NaN
1019998,"Linda's Silicone Stove Gap Covers (2 Pack), He...",NaN,Gabby S,5.0 out of 5 stars,Reviewed in the United States on 22 October 2018,"Size Name: 21""Colour Name: Black",Verified Purchase,\n Easy to install and clean. Haven’t put any...,NaN
1019999,"Linda's Silicone Stove Gap Covers (2 Pack), He...",NaN,Momofthehouse,5.0 out of 5 stars,Reviewed in the United States on 7 December 2020,"Size Name: 21""Colour Name: Black",Verified Purchase,\n It’s exactly as described. Works with my s...,NaN


In [236]:
df.rename(columns = {"Text":"Product", "Title":"Title of Review", "View":"Location and Date", "aiconalt":"Individual Review", "View1": "Review", "asizebase": "Helpful"}, inplace= True)

In [237]:
df

,Product,Title of Review,Name,Individual Review,Location and Date,asizemini,State,Review,Helpful
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0 out of 5 stars,Reviewed in the United Kingdom on 18 September...,Size Name: 45 Count (Pack of 1),Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0 out of 5 stars,Reviewed in the United Kingdom on 11 July 2019,Size Name: 45 Count (Pack of 1),Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0 out of 5 stars,Reviewed in the United Kingdom on 8 May 2019,Size Name: 45 Count (Pack of 1),Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0 out of 5 stars,Reviewed in the United Kingdom on 2 March 2018,Size Name: 45 Count (Pack of 1),Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0 out of 5 stars,Reviewed in the United Kingdom on 29 March 2018,Size Name: 45 Count (Pack of 1),Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful
...,...,...,...,...,...,...,...,...,...
1019995,Garnier Hand Repair Intensive Restoring Hand C...,Nothing special,Janice,3.0 out of 5 stars,Reviewed in the United Kingdom on 3 February 2017,NaN,Verified Purchase,"\n Very ordinary product. Initially, the han...",NaN
1019996,"Linda's Silicone Stove Gap Covers (2 Pack), He...",NaN,Tina,5.0 out of 5 stars,Reviewed in the United States on 9 January 2019,"Size Name: 21""Colour Name: Black",Verified Purchase,\n It came really fast. We have some difficul...,NaN
1019997,"Linda's Silicone Stove Gap Covers (2 Pack), He...",NaN,Mainegirl,5.0 out of 5 stars,Reviewed in the United States on 2 February 2020,"Size Name: 21""Colour Name: Black",Verified Purchase,\n Very happy with this purchase. Just what I...,NaN
1019998,"Linda's Silicone Stove Gap Covers (2 Pack), He...",NaN,Gabby S,5.0 out of 5 stars,Reviewed in the United States on 22 October 2018,"Size Name: 21""Colour Name: Black",Verified Purchase,\n Easy to install and clean. Haven’t put any...,NaN


In [238]:
df.isna().sum()

Product                   0
Title of Review      305188
Name                    567
Individual Review       530
Location and Date       530
asizemini            238599
State                 28373
Review                 1273
Helpful              897123
dtype: int64

In [239]:
df.drop(columns="asizemini", inplace=True)

In [240]:
df.dropna(subset=["Review"], inplace=True)

In [241]:
df.isna().sum()

Product                   0
Title of Review      304341
Name                     37
Individual Review         0
Location and Date         0
State                 27824
Review                    0
Helpful              895934
dtype: int64

In [242]:
df.describe()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful
count,1018727,714386,1018690,1018727,1018727,990903,1018727,122793
unique,950,333843,519420,5,27200,1,904448,415
top,Moleskine Classic Ruled Paper Notebook,Five Stars,Amazon Customer,5.0 out of 5 stars,Reviewed in the United Kingdom on 13 December ...,Verified Purchase,\n Good\n,One person found this helpful
freq,8327,47287,69286,724449,1025,990903,4430,68307


In [243]:
df[["Individual Review"]] = df[["Individual Review"]].apply(lambda x: x.str.rstrip(' out of 5 stars'))

In [244]:
df =  df.astype(str)


In [245]:
df.head()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,Reviewed in the United Kingdom on 18 September...,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,Reviewed in the United Kingdom on 11 July 2019,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,Reviewed in the United Kingdom on 8 May 2019,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,Reviewed in the United Kingdom on 2 March 2018,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,Reviewed in the United Kingdom on 29 March 2018,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful


In [246]:
df['Location and Date'] = df['Location and Date'].apply(lambda x: x.lstrip('Reviewed in the '))

In [247]:
df.head()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom on 18 September 2017,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom on 11 July 2019,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom on 8 May 2019,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom on 2 March 2018,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom on 29 March 2018,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful


In [248]:
df["Helpful"] = df['Helpful'].apply(lambda x: x.replace(',',''))

In [249]:
df.head()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom on 18 September 2017,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom on 11 July 2019,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom on 8 May 2019,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom on 2 March 2018,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom on 29 March 2018,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful


In [250]:
df["Date"] = df["Location and Date"]

In [251]:
df["Date"] = df["Date"].apply(lambda x: x.split(' on ')[-1].strip())

In [252]:
df.head()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful,Date
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom on 18 September 2017,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful,18 September 2017
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom on 11 July 2019,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful,11 July 2019
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom on 8 May 2019,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful,8 May 2019
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom on 2 March 2018,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful,2 March 2018
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom on 29 March 2018,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful,29 March 2018


In [253]:
df["Location and Date"] = df["Location and Date"].apply(lambda x: x.split(' on ')[0].strip())

In [254]:
df.head()

,Product,Title of Review,Name,Individual Review,Location and Date,State,Review,Helpful,Date
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful,18 September 2017
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful,11 July 2019
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful,8 May 2019
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful,2 March 2018
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful,29 March 2018


In [255]:
df.rename(columns={"Location and Date": "Location"}, inplace = True)

In [256]:
df.head()

,Product,Title of Review,Name,Individual Review,Location,State,Review,Helpful,Date
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful,18 September 2017
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful,11 July 2019
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful,8 May 2019
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful,2 March 2018
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful,29 March 2018


In [257]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1018727 entries, 0 to 1019999
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   Product            1018727 non-null  object
 1   Title of Review    1018727 non-null  object
 2   Name               1018727 non-null  object
 3   Individual Review  1018727 non-null  object
 4   Location           1018727 non-null  object
 5   State              1018727 non-null  object
 6   Review             1018727 non-null  object
 7   Helpful            1018727 non-null  object
 8   Date               1018727 non-null  object
dtypes: object(9)
memory usage: 77.7+ MB


In [258]:
df[["day","month", "year"]] = df["Date"].apply(lambda x: pd.Series(x.split(' ')))

In [259]:
df["Date"] = pd.to_datetime(df['year'].astype(str)  + df['month'] + df["day"], format='%Y%B%d')

In [260]:
df.head()

,Product,Title of Review,Name,Individual Review,Location,State,Review,Helpful,Date,day,month,year
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593 people found this helpful,2017-09-18,18,September,2017
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom,Verified Purchase,"\n I've been ordering these for about a year,...",242 people found this helpful,2019-07-11,11,July,2019
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom,Verified Purchase,\n Ibought these after buying similar ones pr...,209 people found this helpful,2019-05-08,8,May,2019
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom,Verified Purchase,\n I have to admit that I just don't like And...,34 people found this helpful,2018-03-02,2,March,2018
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom,Verified Purchase,\n Not really a bargain as it comes a lot les...,216 people found this helpful,2018-03-29,29,March,2018


In [261]:
df.drop(columns = ["day", "month", "year"], inplace= True)

In [262]:
df["Helpful"].value_counts()

nan                              895934
One person found this helpful     68307
2 people found this helpful       18811
3 people found this helpful        8618
4 people found this helpful        4953
                                  ...  
504 people found this helpful         1
341 people found this helpful         1
565 people found this helpful         1
601 people found this helpful         1
475 people found this helpful         1
Name: Helpful, Length: 416, dtype: int64

In [263]:
df["Helpful"] = df["Helpful"].apply(lambda x: x.rstrip('people found this helpful'))

In [264]:
df["Helpful"].value_counts()

na         895934
One per     68307
2           18811
3            8618
4            4953
            ...  
529             1
420             1
611             1
891             1
213             1
Name: Helpful, Length: 416, dtype: int64

In [265]:
df["Helpful"].replace(['One per', "na"], [1, 0], inplace = True)

In [266]:
df.head()

,Product,Title of Review,Name,Individual Review,Location,State,Review,Helpful,Date
0,Andrex Gentle Clean Toilet Tissue,Fun Size C Wafer Thin Andrex,DM Bunny,1.0,United Kingdom,Verified Purchase,"How easily people are fooled.\n\nThese ""Andrex...",593,2017-09-18
1,Andrex Gentle Clean Toilet Tissue,"They've shrunk..half the size, same price...",H.,1.0,United Kingdom,Verified Purchase,"\n I've been ordering these for about a year,...",242,2019-07-11
2,Andrex Gentle Clean Toilet Tissue,"Not the same quality as norm,al Andrex",Adrian Lingard,1.0,United Kingdom,Verified Purchase,\n Ibought these after buying similar ones pr...,209,2019-05-08
3,Andrex Gentle Clean Toilet Tissue,Bargain price - just be careful to get rid of ...,Mogwai,3.0,United Kingdom,Verified Purchase,\n I have to admit that I just don't like And...,34,2018-03-02
4,Andrex Gentle Clean Toilet Tissue,Not really a bargain,Magdalene,1.0,United Kingdom,Verified Purchase,\n Not really a bargain as it comes a lot les...,216,2018-03-29


In [267]:
df[["Product", "Title of Review", "Name", "Location", "State", "Review"]].astype(str).dtypes
df[["Individual Review"]].astype(float)
df[["Helpful"]].astype(int)

,Helpful
0,593
1,242
2,209
3,34
4,216
...,...
1019995,0
1019996,0
1019997,0
1019998,0


In [268]:
df["Location"].value_counts()

United Kingdom    714459
United States      76238
Italy              65322
Germany            59078
Spain              43261
France             31253
Canada             16841
Mexico              5955
India               2268
Japan               1772
Australia           1025
Brazil               798
Netherlands          227
Sweden               153
Singapore             52
Poland                25
Name: Location, dtype: int64

In [269]:
df.describe()

<ipython-input-269-ea8415b8a3ee>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,Product,Title of Review,Name,Individual Review,Location,State,Review,Helpful,Date
count,1018727,1018727,1018727,1018727,1018727,1018727,1018727,1018727.0,1018727
unique,950,333844,519421,5,16,2,904448,416.0,4677
top,Moleskine Classic Ruled Paper Notebook,nan,Amazon Customer,5.0,United Kingdom,Verified Purchase,\n Good\n,0.0,2020-12-13 00:00:00
freq,8327,304341,69286,724449,714459,990903,4430,895934.0,1269
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003-12-15 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-13 00:00:00


In [270]:
(df == "NaN").all(axis=0)

Product              False
Title of Review      False
Name                 False
Individual Review    False
Location             False
State                False
Review               False
Helpful              False
Date                 False
dtype: bool

In [300]:
#greater than the start date and smaller than the end date
df_covd_dates = df[(df["Date"] > "2019-12-01") & (df['Date'] <= "2020-08-01")]

In [301]:
df_covd_dates.describe()

<ipython-input-301-41ec728bacd6>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_covd_dates.describe()


,Product,Title of Review,Name,Individual Review,Location,State,Review,Helpful,Date
count,155836,155836,155836,155836,155836,155836,155836,155836.0,155836
unique,738,60053,97059,5,14,2,141479,177.0,244
top,"Gritin Resistance Bands, [Set of 5] Skin-Frien...",nan,Amazon Customer,5.0,United Kingdom,Verified Purchase,\n Good\n,0.0,2020-01-02 00:00:00
freq,1579,48023,10532,113404,107830,153357,751,139911.0,1002
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-02 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-01 00:00:00


In [302]:
(df_covd_dates["Review"] == "NaN").value_counts()

False    155836
Name: Review, dtype: int64

In [303]:
df_covd_dates["Review"].value_counts()

\n  Good\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 751
\n  Great\n                                                                                                                                                                                              

In [304]:
df_covd_dates["Location"].value_counts()

United Kingdom    107830
United States      11639
Italy              11279
Germany             8516
Spain               7685
France              4578
Canada              2558
Mexico               997
India                308
Japan                240
Australia            138
Brazil                52
Singapore             10
Netherlands            6
Name: Location, dtype: int64

### Drop non English speaking countries

In [305]:
df_covd_dates = df_covd_dates.loc[df_covd_dates["Location"].isin(["United Kingdom", "United States", "Canada", "India", "Australia"])]

In [306]:
df_covd_dates["Location"].value_counts()

United Kingdom    107830
United States      11639
Canada              2558
India                308
Australia            138
Name: Location, dtype: int64

In [307]:
df_covd_dates.dtypes


Product                      object
Title of Review              object
Name                         object
Individual Review            object
Location                     object
State                        object
Review                       object
Helpful                      object
Date                 datetime64[ns]
dtype: object

In [308]:
df_covd_dates

,Product,Title of Review,Name,Individual Review,Location,State,Review,Helpful,Date
10,Andrex Gentle Clean Toilet Tissue,Overpriced nonsense,Mr R H Angus,2.0,United Kingdom,Verified Purchase,\n Why the change of price for those of us wh...,2,2020-03-19
15,Andrex Gentle Clean Toilet Tissue,Absolutely not worth the money,yc chai,1.0,United Kingdom,Verified Purchase,I never thought that I still need to read any ...,3,2020-05-30
21,Andrex Gentle Clean Toilet Tissue,CON,Customer in York,1.0,United Kingdom,Verified Purchase,\n We brought Andrex multi on a few occasions...,2,2020-01-05
24,Andrex Gentle Clean Toilet Tissue,Sad sad sad,El,1.0,United Kingdom,Verified Purchase,"\n Disappointed in these thinned out, less de...",2,2020-03-09
28,Andrex Gentle Clean Toilet Tissue,Beware - inferior quality,Kenkonti,1.0,United Kingdom,Verified Purchase,\n This product sums up the pitfalls of buyin...,2,2020-06-24
...,...,...,...,...,...,...,...,...,...
1019983,AnySharp Knife Sharpener with PowerGrip,Make your own razors,Cupid Stunt,5.0,United Kingdom,Verified Purchase,"\n This is proper, will take metal shavings o...",0,2020-01-24
1019992,Garnier Hand Repair Intensive Restoring Hand C...,Ideal product,Amazon Customer,5.0,United Kingdom,Verified Purchase,\n A****\n,0,2020-05-09
1019993,Garnier Hand Repair Intensive Restoring Hand C...,Good and Cheap,Paul Fadur,5.0,United Kingdom,Verified Purchase,\n Good and Cheap for value\n,0,2020-02-22
1019994,Garnier Hand Repair Intensive Restoring Hand C...,Repair dry hand very well,Paul Bell,5.0,United Kingdom,Verified Purchase,\n Very good hand cream\n,0,2020-02-18


In [309]:
df_covd_dates = df_covd_dates.drop(columns = ["Title of Review", "Name", "State"])
df_covd_dates

,Product,Individual Review,Location,Review,Helpful,Date
10,Andrex Gentle Clean Toilet Tissue,2.0,United Kingdom,\n Why the change of price for those of us wh...,2,2020-03-19
15,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,I never thought that I still need to read any ...,3,2020-05-30
21,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,\n We brought Andrex multi on a few occasions...,2,2020-01-05
24,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,"\n Disappointed in these thinned out, less de...",2,2020-03-09
28,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,\n This product sums up the pitfalls of buyin...,2,2020-06-24
...,...,...,...,...,...,...
1019983,AnySharp Knife Sharpener with PowerGrip,5.0,United Kingdom,"\n This is proper, will take metal shavings o...",0,2020-01-24
1019992,Garnier Hand Repair Intensive Restoring Hand C...,5.0,United Kingdom,\n A****\n,0,2020-05-09
1019993,Garnier Hand Repair Intensive Restoring Hand C...,5.0,United Kingdom,\n Good and Cheap for value\n,0,2020-02-22
1019994,Garnier Hand Repair Intensive Restoring Hand C...,5.0,United Kingdom,\n Very good hand cream\n,0,2020-02-18


In [310]:
df_covd_dates.reset_index(drop=True, inplace=True)
df_covd_dates

,Product,Individual Review,Location,Review,Helpful,Date
0,Andrex Gentle Clean Toilet Tissue,2.0,United Kingdom,\n Why the change of price for those of us wh...,2,2020-03-19
1,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,I never thought that I still need to read any ...,3,2020-05-30
2,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,\n We brought Andrex multi on a few occasions...,2,2020-01-05
3,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,"\n Disappointed in these thinned out, less de...",2,2020-03-09
4,Andrex Gentle Clean Toilet Tissue,1.0,United Kingdom,\n This product sums up the pitfalls of buyin...,2,2020-06-24
...,...,...,...,...,...,...
122468,AnySharp Knife Sharpener with PowerGrip,5.0,United Kingdom,"\n This is proper, will take metal shavings o...",0,2020-01-24
122469,Garnier Hand Repair Intensive Restoring Hand C...,5.0,United Kingdom,\n A****\n,0,2020-05-09
122470,Garnier Hand Repair Intensive Restoring Hand C...,5.0,United Kingdom,\n Good and Cheap for value\n,0,2020-02-22
122471,Garnier Hand Repair Intensive Restoring Hand C...,5.0,United Kingdom,\n Very good hand cream\n,0,2020-02-18


In [311]:
df_covd_dates.insert(1, 'Category', 'nan')
df_covd_dates.insert(2, 'Price', 0)
df_covd_dates = df_covd_dates[["Product", "Category", "Price", "Individual Review", "Location", "Date", "Review", "Helpful"]]
df_covd_dates

,Product,Category,Price,Individual Review,Location,Date,Review,Helpful
0,Andrex Gentle Clean Toilet Tissue,nan,0,2.0,United Kingdom,2020-03-19,\n Why the change of price for those of us wh...,2
1,Andrex Gentle Clean Toilet Tissue,nan,0,1.0,United Kingdom,2020-05-30,I never thought that I still need to read any ...,3
2,Andrex Gentle Clean Toilet Tissue,nan,0,1.0,United Kingdom,2020-01-05,\n We brought Andrex multi on a few occasions...,2
3,Andrex Gentle Clean Toilet Tissue,nan,0,1.0,United Kingdom,2020-03-09,"\n Disappointed in these thinned out, less de...",2
4,Andrex Gentle Clean Toilet Tissue,nan,0,1.0,United Kingdom,2020-06-24,\n This product sums up the pitfalls of buyin...,2
...,...,...,...,...,...,...,...,...
122468,AnySharp Knife Sharpener with PowerGrip,nan,0,5.0,United Kingdom,2020-01-24,"\n This is proper, will take metal shavings o...",0
122469,Garnier Hand Repair Intensive Restoring Hand C...,nan,0,5.0,United Kingdom,2020-05-09,\n A****\n,0
122470,Garnier Hand Repair Intensive Restoring Hand C...,nan,0,5.0,United Kingdom,2020-02-22,\n Good and Cheap for value\n,0
122471,Garnier Hand Repair Intensive Restoring Hand C...,nan,0,5.0,United Kingdom,2020-02-18,\n Very good hand cream\n,0


In [312]:
df_covd_dates.isna().value_counts()

Product  Category  Price  Individual Review  Location  Date   Review  Helpful
False    False     False  False              False     False  False   False      122473
dtype: int64

## Merge info from other data set

In [313]:
product_category["Product"].value_counts().unique

<bound method Series.unique of Fire 7 Kids tablet | for ages 3-7 | 7" Display, 16 GB | Blue Kid-Proof Case                                                                                                4
Fire HD 10 Kids Pro tablet | for ages 6-12 | 10.1", 1080p Full HD, 32 GB | Black Kid-Friendly Case                                                                         4
Fire HD 8 Kids tablet | for ages 3-7 | 8" HD display, 32 GB | Blue Kid-Proof Case                                                                                          4
Echo Dot (4th generation) | Smart speaker with Alexa | Twilight Blue                                                                                                       3
Command Clear Medium Wire Toggle CL065-10, 10 Hooks, 12 Strips                                                                                                             3
                                                                                                        

In [390]:
df_covd_dates["Product"].value_counts()

VeraCosy Men's Two-Tone Memory Foam Slippers                                                                                                                                                  1437
Gritin Resistance Bands, [Set of 5] Skin-Friendly Resistance Fitness Exercise Loop Bands with 5 Different Resistance Levels - Carrying Case Included - Ideal for Home, Gym, Yoga, Training    1320
KROSER Laptop Backpack School Travel Backpack Stylish Doctor Bag 15.6 Inch Water Repellent College Casual Daypack with USB Port Business Work Computer Bag for Men/Women-Grey                 1258
RENPHO Body Fat Scale Bluetooth, Digital Body Weight Bathroom Scales Weighing Scale Smart BMI Scales, Body Composition Monitors with Smartphone App                                           1234
RockDove Men's Original Two-Tone Memory Foam Slipper                                                                                                                                          1119
                         

In [372]:
df_covd_dates.isna().sum()

Product              0
Category             0
Price                0
Individual Review    0
Location             0
Date                 0
Review               0
Helpful              0
dtype: int64

In [391]:
df_covd_dates_merged = df_covd_dates.merge(product_category[["Product", "Category", "Price"]], on="Product", how = "left")

In [413]:
df_covd_dates_merged

,Product,Category_x,Price_x,Individual Review,Location,Date,Review,Helpful,Category_y,Price_y
0,Andrex Gentle Clean Toilet Tissue,nan,0,2.0,United Kingdom,2020-03-19,\n Why the change of price for those of us wh...,2,NaN,NaN
1,Andrex Gentle Clean Toilet Tissue,nan,0,1.0,United Kingdom,2020-05-30,I never thought that I still need to read any ...,3,NaN,NaN
2,Andrex Gentle Clean Toilet Tissue,nan,0,1.0,United Kingdom,2020-01-05,\n We brought Andrex multi on a few occasions...,2,NaN,NaN
3,Andrex Gentle Clean Toilet Tissue,nan,0,1.0,United Kingdom,2020-03-09,"\n Disappointed in these thinned out, less de...",2,NaN,NaN
4,Andrex Gentle Clean Toilet Tissue,nan,0,1.0,United Kingdom,2020-06-24,\n This product sums up the pitfalls of buyin...,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
123705,AnySharp Knife Sharpener with PowerGrip,nan,0,5.0,United Kingdom,2020-01-24,"\n This is proper, will take metal shavings o...",0,NaN,NaN
123706,Garnier Hand Repair Intensive Restoring Hand C...,nan,0,5.0,United Kingdom,2020-05-09,\n A****\n,0,beauty,1.99
123707,Garnier Hand Repair Intensive Restoring Hand C...,nan,0,5.0,United Kingdom,2020-02-22,\n Good and Cheap for value\n,0,beauty,1.99
123708,Garnier Hand Repair Intensive Restoring Hand C...,nan,0,5.0,United Kingdom,2020-02-18,\n Very good hand cream\n,0,beauty,1.99


In [441]:
df1 = product_category
df2 = df_covd_dates

In [442]:
df1

,Product,Category,Price
0,Andrex Gentle Clean Toilet Rolls - 45 Toilet R...,grocery,21.09
1,Tassimo Kenco Americano Grande Coffee Pods (Pa...,grocery,15.76
2,DYLON Washing Machine Fabric Dye Pod for Cloth...,grocery,5.00
3,Tassimo Costa Gingerbread Latte Coffee Pods (P...,grocery,22.48
4,Lindt Lindor Milk Chocolate Truffles Box with ...,grocery,3.99
...,...,...,...
2247,Sea Monkeys DI23230 Sea Monkeys Magic Castle,kids,7.50
2248,Unique Party 9054 - Blue Hair Spray,kids,2.45
2249,Nuby Octopus Floating Bath Toy,kids,8.25
2250,LEGO 43201 Disney Isabela’s Magical Door Build...,kids,17.99


In [443]:
df2

,Product,Category,Price,Individual Review,Location,Date,Review,Helpful
0,Andrex Gentle Clean Toilet Tissue,nan,0,2.0,United Kingdom,2020-03-19,\n Why the change of price for those of us wh...,2
1,Andrex Gentle Clean Toilet Tissue,nan,0,1.0,United Kingdom,2020-05-30,I never thought that I still need to read any ...,3
2,Andrex Gentle Clean Toilet Tissue,nan,0,1.0,United Kingdom,2020-01-05,\n We brought Andrex multi on a few occasions...,2
3,Andrex Gentle Clean Toilet Tissue,nan,0,1.0,United Kingdom,2020-03-09,"\n Disappointed in these thinned out, less de...",2
4,Andrex Gentle Clean Toilet Tissue,nan,0,1.0,United Kingdom,2020-06-24,\n This product sums up the pitfalls of buyin...,2
...,...,...,...,...,...,...,...,...
122468,AnySharp Knife Sharpener with PowerGrip,nan,0,5.0,United Kingdom,2020-01-24,"\n This is proper, will take metal shavings o...",0
122469,Garnier Hand Repair Intensive Restoring Hand C...,nan,0,5.0,United Kingdom,2020-05-09,\n A****\n,0
122470,Garnier Hand Repair Intensive Restoring Hand C...,nan,0,5.0,United Kingdom,2020-02-22,\n Good and Cheap for value\n,0
122471,Garnier Hand Repair Intensive Restoring Hand C...,nan,0,5.0,United Kingdom,2020-02-18,\n Very good hand cream\n,0


In [448]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [476]:
# r = fuzzy_merge(q, df1, 'Product', 'Product', threshold=80)

In [477]:
r

,Product,matches
0,Andrex Gentle Clean Toilet Tissue,Andrex Gentle Clean Toilet Rolls - 45 Toilet R...
1,"Celebrations Chocolate Bulk Box, Chocolate Gif...","Celebrations Chocolate Bulk Box, Chocolate Gif..."
2,Eco Bag 50 Heavy Duty Refuse Sacks,"Eco Bag 50 Heavy Duty Refuse Sacks, Black, 1x,..."
3,SonicScrubber Household Electrical Cleaning Br...,SonicScrubber Household Electrical Cleaning Br...
4,Nescafe Dolce Gusto Flat White Coffee Pods (Pa...,Nescafe Dolce Gusto Flat White Coffee Pods (Pa...
...,...,...
729,CIARRA Touch Control Chimney Cooker Hood 650 m...,CIARRA CBCS6102 Class A++ Cooker Hood 650 m³/h...
730,"Just for men Moustache & Beard Dye , Eliminate...",Just for men Moustache & Beard Medium Brown Dy...
731,Peek 33300 Metal Polish Paste 1L,Peek 33300 Metal Polish Paste 1L & Y048 33200 ...
732,Remington D3010 2000W Power Dry Hair Dryer,"Buster Bathroom Plughole Unblocker, Dissolves ..."


In [479]:
q

,Product,matches
0,Andrex Gentle Clean Toilet Tissue,Andrex Gentle Clean Toilet Rolls - 45 Toilet R...
1,"Celebrations Chocolate Bulk Box, Chocolate Gif...","Celebrations Chocolate Bulk Box, Chocolate Gif..."
2,Eco Bag 50 Heavy Duty Refuse Sacks,"Eco Bag 50 Heavy Duty Refuse Sacks, Black, 1x,..."
3,SonicScrubber Household Electrical Cleaning Br...,SonicScrubber Household Electrical Cleaning Br...
4,Nescafe Dolce Gusto Flat White Coffee Pods (Pa...,Nescafe Dolce Gusto Flat White Coffee Pods (Pa...
...,...,...
729,CIARRA Touch Control Chimney Cooker Hood 650 m...,CIARRA CBCS6102 Class A++ Cooker Hood 650 m³/h...
730,"Just for men Moustache & Beard Dye , Eliminate...",Just for men Moustache & Beard Medium Brown Dy...
731,Peek 33300 Metal Polish Paste 1L,Peek 33300 Metal Polish Paste 1L & Y048 33200 ...
732,Remington D3010 2000W Power Dry Hair Dryer,"Buster Bathroom Plughole Unblocker, Dissolves ..."


In [472]:
df2 = df_covd_dates
df2 = df2.drop(columns= (["Category", "Price",	"Individual Review", "Location", "Date", "Review", "Helpful"]))
q = df2["Product"].unique()
q = pd.DataFrame(q, columns=["Product"])
q

,Product
0,Andrex Gentle Clean Toilet Tissue
1,"Celebrations Chocolate Bulk Box, Chocolate Gif..."
2,Eco Bag 50 Heavy Duty Refuse Sacks
3,SonicScrubber Household Electrical Cleaning Br...
4,Nescafe Dolce Gusto Flat White Coffee Pods (Pa...
...,...
729,CIARRA Touch Control Chimney Cooker Hood 650 m...
730,"Just for men Moustache & Beard Dye , Eliminate..."
731,Peek 33300 Metal Polish Paste 1L
732,Remington D3010 2000W Power Dry Hair Dryer


In [475]:
df1

,Product,Category,Price
0,Andrex Gentle Clean Toilet Rolls - 45 Toilet R...,grocery,21.09
1,Tassimo Kenco Americano Grande Coffee Pods (Pa...,grocery,15.76
2,DYLON Washing Machine Fabric Dye Pod for Cloth...,grocery,5.00
3,Tassimo Costa Gingerbread Latte Coffee Pods (P...,grocery,22.48
4,Lindt Lindor Milk Chocolate Truffles Box with ...,grocery,3.99
...,...,...,...
2247,Sea Monkeys DI23230 Sea Monkeys Magic Castle,kids,7.50
2248,Unique Party 9054 - Blue Hair Spray,kids,2.45
2249,Nuby Octopus Floating Bath Toy,kids,8.25
2250,LEGO 43201 Disney Isabela’s Magical Door Build...,kids,17.99


In [316]:
#for idx, chunk in enumerate(np.array_split(df_covd_dates, 3)):
#    chunk.to_csv(f'../Cleaned_&_Sentimental_csv/best_products_for_sentimental_{idx}.csv', index=False)